# Kaznatcheev, et al. (2017)

Consider a cell interacting with $n$ nearby cells. Let $b_a$ be the benefit per unit of acidification. Define
$$A_n(k)=\frac{b_a k}{n+1}$$ to be the benefit due to acidification if $k$ cells produce acid. Similarly, let
$b_v$ be the benefit from oxygen per unit of vascularization, and define $$V_n(k)=\frac{b_v k}{n+1}$$ to be
the benefit due to vascularization if $k$ cells are (over) producing VEGF. Let $c$ be the cost of VEGF production.

* The GLY strategy produces acid and does not use oxygen. It does not (over) produce VEGF. The payoff is $A_n(n_G+1)$ if there are $n_G$ other GLY cells.
* The VOP strategy uses oxygen and does not produce acid. It (over) produces VEGF to increase vascularization. The payoff is $A_n(n_G)+V_{n-n_G}(n_V+1)-c$ if there are $n_G$ GLY cells and $n_V$ other VOP cells.
* The DEF strategy uses oxygen and does not produce acid, and it does not (over) produce VEGF. The payoff is $A_n(n_G)+V_{n-n_G}(n_V)$.

If $x_G$ is the proportion of GLY cells in the population, then the expected fitness of the glycolytic cells is
$$w_G=\langle A_n(n_G+1)\rangle_{n_G\sim\textbf{B}_n(x_G)}=\left\langle\frac{b_a(n_G+1)}{n+1}\right\rangle_{n_G\sim\textbf{B}_n(x_G)},$$
where $\langle\cdot\rangle_{n\sim\textbf{D}}$ computes the average over the random variable $n$ sampled from the distribution $\textbf{D}$;
in this case, $\textbf{B}_n(x_G)$ is the binomial distribution with $n$ trials and probability of success $x_G$.
Similarly, if $x_V$ and $x_D=1-x_G-x_V$ are the proportions of VOP and DEF cells, respectively, then the expected fitness values of those corresponding populations are
$$w_V=\left\langle\frac{b_a n_G}{n+1}\right\rangle_{n_G\sim\textbf{B}_n(x_G)}
+\left\langle\frac{b_v(n_V+1)}{n-n_G+1}\right\rangle_{n_G,n_V\sim\textbf{M}_n(x_G,x_V)}-c$$
and
$$w_D=\left\langle\frac{b_a n_G}{n+1}\right\rangle_{n_G\sim\textbf{B}_n(x_G)}
+\left\langle\frac{b_v n_V}{n-n_G+1}\right\rangle_{n_G,n_V\sim\textbf{M}_n(x_G,x_V)},$$
where $\textbf{M}_n(x_G,x_V)$ is the multinomial distribution with $n$ trials and probabilities of the first and second outcomes $x_G$ and $x_V$.
The expected fitness of the entire population is then $\langle w\rangle=x_Gw_G+x_Vw_V+x_Dw_D$.


The evolutionary dynamics of the population are given by the replicator equations:
$$\dot x_G=x_G(w_G-\langle w\rangle)$$
$$\dot x_V=x_V(w_V-\langle w\rangle)$$
$$\dot x_D=x_D(w_D-\langle w\rangle)$$


## Factoring to Reduced (2D) Coordinates

Let $p=x_G$ be the proportion of GLY cells, and $q=\frac{x_V}{x_V+x_D}$ be the proportion of aerobic (non-glycolytic) cells that follow the VOP strategy. The dynamics may then be rewritten as
$$\dot p=p(1-p)(w_G-q w_V-(1-q)w_D)$$
$$\dot q=q(1-q)(w_V-w_D).$$

Using properties of binomial and multinomial distributions, we may evaluate the expected values to find the following equivalent equations:
$$\dot p=p(1-p)\left(\frac{b_a}{n+1}-q(b_v-c)\right)$$
$$\dot q=q(1-q)\left(\frac{b_v}{n+1}\left(\sum_{k=0}^n p^k\right)-c\right)$$

## Gluzman Code

In [4]:
// Model parameters
val d_max = 3.0
val sigma = 0.01
val ba = 2.5
val bv = 2.0
val c = 1.0
val n_neigh = 4
val fb = math.sqrt(1e-3)
val rb = fb

// Discretization parameters
val n = 9000
val h = 1.0 / n
val iter_tol = 1e-4
val hugeVal = 100000
val tinyVal = 1e-10

d_max: Double = 3.0
sigma: Double = 0.01
ba: Double = 2.5
bv: Double = 2.0
c: Double = 1.0
n_neigh: Int = 4
fb: Double = 0.03162277660168379
rb: Double = 0.03162277660168379
n: Int = 9000
h: Double = 1.1111111111111112E-4
iter_tol: Double = 1.0E-4
hugeVal: Int = 100000
tinyVal: Double = 1.0E-10

The `u(i, j)` matrix gives the value function $u(x_D, x_G)$, where $x_D = i\cdot h$ and $x_G = j\cdot h$ (and $x_V = 1 - x_D - x_G$).
We have $0\leq x_D, x_G$ and $x_D + x_G\leq 1$.

If $x_G < r_b$, then _recovery_; if $x_G > 1 - f_b$, then _failure_.

In [5]:
def u_initiation(n: Int): Array[Array[Double]] = Array.tabulate(n+1, n+1) {(i, j) =>
  if (i + j <= n) { // x_D + x_G <= 1
    if (j * h < rb) { // x_G < rb
      0
    } else {
      hugeVal
    }
  } else {
    Double.NaN
  }
}

// (x_D, x_G) coordinates
type State = (Double, Double)

// Instantaneous cost
def K(x: State, d: Double): Double = d + sigma

defined function u_initiation
defined type State
defined function K

Compute $\langle\dot x_D, \dot x_G\rangle$, given $\langle x_D, x_G\rangle$ and $d$.
Since $x_D = (1-p)(1-q)$, $\dot x_D = -\dot q(1 - p) -\dot p(1 - q)$.

Note that when $x_G=0$ (bottom edge of triangle, in recovery zone), $\dot x_G=0$.
Similarly, when $x_D=0$ (left edge) or $x_V=0$ (right edge), $\dot x_D=0$. Therefore, the direction of movement never points outside the triangle.

In [7]:
def f(x: State, d: Double): State = {
  val (x_D, x_G) = x
  val x_V = 1 - x_D - x_G
  val p = x_G
  val q = x_V / (x_V + x_D)
  
  // Since p < 1.0, this is (1 - math.pow(p, n_neigh + 1)) / (1 - p):
  val sum_p = ((0 to n_neigh) map {z => math.pow(p, z)}).sum
  
  val dq = q * (1 - q) * (bv / (n_neigh + 1) * sum_p - c)
  val dp = p * (1 - p) * (ba / (n_neigh + 1) - q * (bv - c) - d)
  
  (-dq * (1 - p) - dp * (1 - q), dp)
}

defined function f

In [11]:
def tau_func(x: State, d: Double, i: Int, j: Int): Double = {
  val (x1, x2) = x
  val (dx1, dx2) = f(x, d)
  assert(dx1*dx1 + dx2*dx2 > 0)
  
  val (sdx1, sdx2) = (math.signum(dx1), math.signum(dx2))
  
  if (dx1 == 0) {
    h / math.abs(dx2)
  } else if (dx2 == 0) {
    h / math.abs(dx1)
  } else {
    val (dj1, di2) = if (dx1 * dx2 > 0) {
      (0.0, 0.0)
    } else if (math.abs(dx2) > math.abs(dx1)) {
      (sdx2, 0.0)
    } else {
      (0.0, sdx1)
    }
    
    val (x11, x12, x21, x22) = ((i + sdx1) * h, (j + dj1) * h, (i + di2) * h, (j + sdx2) * h)
    
    val k1 = x21 - x11
    val k2 = x12 - x22
    val kc = -(x12 * k1 + x11 * k2)
    -(kc + k1 * x2 + k2 * x1) / (k1 * dx2 + k2 * dx1)
  }
} ensuring {y => !y.isNaN && !y.isInfinite && y > 0}

defined function tau_func

Alternate computation of neighboring interpolation points. Given grid point `(i, j)` (corresponding to state $x$) and control `d`, return grid points `(i1, j1)` and `(i2, j2)` (corresponding to neighboring states $x_1$ and $x_2$), plus interpolation fraction $\gamma$ and time step $\tau$ such that $\tilde x=(1-\gamma)x_1 + \gamma x_2$ is the point $x+\tau f(x,d)$.

In [72]:
def neighbors(i: Int, j: Int, d: Double): ((Int, Int), (Int, Int), Double, Double) = {
  val x = (i * h, j * h)
  val (dx_D, dx_G) = f(x, d)
  assert(dx_D*dx_D + dx_G*dx_G > 0)
  
  val ((i1, j1), (i2, j2)) =
  if (dx_D >= 0 && dx_G >= 0) {
    ((i, j+1), (i+1, j))
  } else if (dx_D >= 0 && dx_G >= -dx_D) {
    ((i+1, j), (i+1, j-1))
  } else if (dx_D >= 0) {
    ((i+1, j-1), (i, j-1))
  } else if (dx_G <= 0) {
    ((i, j-1), (i-1, j))
  } else if (dx_G <= -dx_D) {
    ((i-1, j), (i-1, j+1))
  } else {
    ((i-1, j+1), (i, j+1))
  }
  
  val (x_D, x_G) = x
  val (x1_D, x1_G) = (i1 * h, j1 * h)
  val (x2_D, x2_G) = (i2 * h, j2 * h)
  
  // Find intersection xtilde of line through x1 and x2
  // with ray based at x having direction f(x, d)
  // See https://en.wikipedia.org/wiki/Line%E2%80%93line_intersection
  val det = (x1_G - x2_G) * dx_D - (x1_D - x2_D) * dx_G
  val gamma = ((x1_G - x_G) * dx_D - (x1_D - x_D) * dx_G) / det
  val tau = ((x1_G - x2_G) * (x1_D - x_D) - (x1_D - x2_D) * (x1_G - x_G)) / det
  
  ((i1, j1), (i2, j2), gamma, tau)
}

defined function neighbors

In [71]:
import scala.util.Random
val i = Random.nextInt(n + 1)
val j = Random.nextInt(n + 1 - i)
val d = d_max * Random.nextInt(2)
val tau1 = tau_func((i * h, j * h), d, i, j)
val (_, _, _, tau2) = neighbors(i, j, d)
println(tau1 - tau2)

-1.1752751549742868E-16


import scala.util.Random

i: Int = 5516
j: Int = 4107
d: Double = 0.0
tau1: Double = 5.634188013635888E-4
tau2: Double = 5.634188013637063E-4